In [126]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [174]:
reading_file = open("SMSSpam.csv", "r")
# clean the file
new_file_content = ""
for line in reading_file:
    stripped_line = line.strip()
    new_line = stripped_line.replace("\"", "")
    new_file_content += new_line +"\n"
reading_file.close()

writing_file = open("SMSSpam.csv", "w")
writing_file.write(new_file_content)
writing_file.close()

# reference and clean the data
df = pd.read_csv('SMSSpam.csv', usecols=[1,2]).dropna()
dfclean = df.rename(columns={"message;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;": "messages"})
messages = dfclean["messages"]
# get message length
tempLen = []
for x in messages:
    tempLen.append(len(x))
# reference specific words in messages
dfclean["length"] = tempLen
dfclean["free"] = dfclean["messages"].str.contains("free", case=False)
dfclean["winner"] = dfclean["messages"].str.contains("winner", case=False)
dfclean["subscription"] = dfclean["messages"].str.contains("subscription", case=False)
dfclean["price"] = dfclean["messages"].str.contains("price", case=False)
dfclean["crazy"] = dfclean["messages"].str.contains("crazy", case=False)
dfclean["dick"] = dfclean["messages"].str.contains("dick", case=False)
dfclean

,type,messages,length,free,winner,subscription,price,crazy,dick
0,ham,Go until jurong point,21,False,False,False,False,False,False
1,ham,Ok lar... Joking wif u oni...;;;;;;;;;;;;;;;;;...,65,False,False,False,False,False,False
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,191,True,False,False,False,False,False
3,ham,U dun say so early hor... U c already then say...,85,False,False,False,False,False,False
4,ham,Nah I don't think he goes to usf,32,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,96,False,False,False,False,False,False
5568,ham,Will ü b going to esplanade fr home?;;;;;;;;;;...,72,False,False,False,False,False,False
5569,ham,Pity,4,False,False,False,False,False,False
5570,ham,The guy did some bitching but I acted like i'd...,161,True,False,False,False,False,False


In [175]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors = 6)

In [176]:
knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=6)

In [177]:
knn.score(X_test, y_test)

0.8908829863603733

In [173]:
from sklearn import metrics
prediction = knn.predict(X_scale)

def confused(Y_test, prediction):
    confusion = metrics.confusion_matrix(Y_test, prediction)
    TP = confusion[1][1]
    TN = confusion[0][0]
    FP = confusion[0][1]
    FN = confusion[1][0]

    print('True Positives:', TP)
    print('True Negatives:', TN)
    print('False Positives:', FP)
    print('False Negatives:', FN)
confused(y, prediction)

True Positives: 4756
True Negatives: 0
False Positives: 1
False Negatives: 0


In [122]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
k_range = range(1, 44)
param_grid = {'n_neighbors' : k_range ,
 'p' : (1,2),
 'weights' : ('uniform','distance'),
 'metric' : ('minkowski', 'chebyshev'),
 }
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy')
grid.fit(X_scale, y)

GridSearchCV(cv=10, estimator=KNeighborsClassifier(),
             param_grid={'metric': ('minkowski', 'chebyshev'),
                         'n_neighbors': range(1, 44), 'p': (1, 2),
                         'weights': ('uniform', 'distance')},
             scoring='accuracy')

In [123]:
grid.best_estimator_

KNeighborsClassifier(n_neighbors=6, p=1)

In [124]:
grid.best_score_

0.8892096017979153

In [141]:
dfclean.head()

,type,messages,length,free,winner,subscription,price
0,ham,Go until jurong point,21,False,False,False,False
1,ham,Ok lar... Joking wif u oni...;;;;;;;;;;;;;;;;;...,65,False,False,False,False
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,191,True,False,False,False
3,ham,U dun say so early hor... U c already then say...,85,False,False,False,False
4,ham,Nah I don't think he goes to usf,32,False,False,False,False


In [163]:
dicted1 = {"length": [20],"free": [False],"winner": [False],"subscription": [False],"price": [False]}
dicted2 = {"length": [24],"free": [True],"winner": [True],"subscription": [False],"price": [False]}
case1 = pd.DataFrame(data=dicted1)
case2 = pd.DataFrame(data=dicted2)
case1.head()

,length,free,winner,subscription,price
0,20,False,False,False,False


In [164]:
knn.predict(case1)

array(['ham'], dtype=object)

In [165]:
knn.predict(case2)

array(['spam'], dtype=object)

In [166]:
knn.predict_proba(case1)

array([[0.5, 0.5]])

In [167]:
knn.predict_proba(case2)

array([[0.16666667, 0.83333333]])